In [129]:
import mysql.connector
import re

def convert_to_int(string):
    clean_string = re.sub('\W+','', string )
    return int(clean_string)


def rt_rating(movie):
    for rating in movie['Ratings']:
        if rating['Source'] == "Rotten Tomatoes":
            rt_v = rating['Value']
            return convert_to_int(rt_v)
        

def connect_to_db():
    cnx = mysql.connector.connect(
    host=config.host, user=config.user, passwd=config.password, database=config.DB_NAME)
    cursor = cnx.cursor()
    return cnx, cursor


def convert_list_from_tuples(tuple_list):
    """The movie titles are retrieved as a list of tupes, 
    return instead a list with just the title from the first half of each tuple"""
    title_list = []
    for item in tuple_list:
        title_list.append(item[0])
    return title_list
    

def get_all_movie_info():
    select_q = """
                SELECT *
                FROM movies
                """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

def get_movie_names():
    name_q = """
            SELECT title
            FROM movies"""
    cnx, c = connect_to_db()
    c.execute(name_q)
    results = c.fetchall()
    cnx.close()
    c.close()
    title_list = convert_list_from_tuples(results) 
    return title_list

titles = get_movie_names()
titles[:5]

['Lock, Stock and Two Smoking Barrels',
 'The Kid',
 'Léon: The Professional',
 'What Ever Happened to Baby Jane?',
 'Taxi Driver']

In [127]:
import requests
import json
import config

def omdb_api(movies):
    """This function retrieves the movie deatils for list of given movies using omdb api"""
    movies_details = []
    for title in movies:
        title.replace(" ", "+")
        url = f'http://www.omdbapi.com/?apikey={config.omdb_api}&t={title}'
        response = requests.get(url)
        movie = response.json()
        movie_dict = {'director': movie['Director'],
                      'boxoffice': movie['BoxOffice'],
                      'rt_rating': rt_rating(movie)}

        movies_details.append(movie_dict)
    
    return movies_details 
      

In [128]:
two = omdb_api(titles[0:2])
two

[{'Director': 'Guy Ritchie', 'BoxOffice': 'N/A', 'RT_Rating': 75.0},
 {'Director': 'Charles Chaplin', 'BoxOffice': 'N/A', 'RT_Rating': 100.0}]

In [117]:
# b = omdb_api(['Guardians of the Galaxy Vol. 2', 'Lock Stock & Two Smoking Barrles'])
for k,v in b[0].items():
    print(f"key: {k}, value: {v}")
# BoxOffice, Director

key: Title, value: Guardians of the Galaxy Vol. 2
key: Year, value: 2017
key: Rated, value: PG-13
key: Released, value: 05 May 2017
key: Runtime, value: 136 min
key: Genre, value: Action, Adventure, Comedy, Sci-Fi
key: Director, value: James Gunn
key: Writer, value: James Gunn, Dan Abnett (based on the Marvel comics by), Andy Lanning (based on the Marvel comics by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Jim Starlin (Gamora and Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Steve Gerber (Howard the Duck created by), Val Mayerik (Howard the Duck created by)
key: Actors, value: Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel
key: Plot, value: The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celest